In [0]:
import logging
import csv
import json
import sys, os, time, glob
import pandas as pd
import requests, urllib, subprocess, wget
from dateutil.relativedelta import relativedelta
from persiantools.jdatetime import JalaliDate
import datetime
#___________________________________________________________________________________________________
logging.basicConfig(format = ' %(asctime)s  -  %(message)s')
logging.disable(level = logging.DEBUG)

Paye_Farabourse = "https://www.sahamyab.com/api/proxy/symbol/stockWatch?v=0.1&namad=&market=4&type=&sector=&page=0&pageSize=2000&sort=&"
Farabourse = "https://www.sahamyab.com/api/proxy/symbol/stockWatch?v=0.1&namad=&market=2&type=&sector=&page=0&pageSize=2000&sort=&"
Bourse = "https://www.sahamyab.com/api/proxy/symbol/stockWatch?v=0.1&namad=&market=1&type=&sector=&page=0&pageSize=2000&sort=&"

links = [Bourse, Farabourse, Paye_Farabourse]
linksName = ["Bourse", "Farabourse", "Paye_Farabourse"]

stock_data_service_providers_with_bURL = {"sahamyab": "https://www.sahamyab.com/guest/tradingview/history?adjustment=&",
                                          "orshan": "https://www.nahayatnegar.com/tv/chart/history?",
                                          "tsetmc": "http://www.tsetmc.com/tsev2/data/Export-txt.aspx?t=i&a=1&b=0&i="}

keys = ['id', 'InsCode', 'type', 'pushId', 'name', 'title', 'corpName', 'sectionName', 'subSectionName',
        'status', 'date', 'q_status', 'tradeCount', 'tradeVolume', 'tradeTotalPrice', 'maxPrice',
        'minPrice', 'firstPrice', 'closingPrice', 'lastPrice', 'yesterdayPrice',
        'sellCountInd', 'sellCountCorp', 'sellVolumeInd', 'sellVolumeCorp', 'buyCountInd',
        'buyCountCorp', 'buyVolumeInd', 'buyVolumeCorp', 'market', 'baseVolume', 'marketValue',
        'totalCount', 'minAllowPrice', 'maxAllowPrice', 'minAllowVolume', 'maxAllowVolume',
        'estimatedEPS', 'PE', 'sectorPE', 'insCode', 'description', 'namad',
        'flow', 'sector', 'subsector', 'tradeDate', 'price', 'tradeValue', 'priceChange',
        'priceChangeClosing', 'indCount', 'indVolume', 'indValue', 'corpCount', 'corpVolume',
        'corpValue', 'priceChange_1', 'priceChange_7', 'priceChange_30', 'priceChange_91', 'priceChange_182',
        'priceChange_365']

stockInfoCol = ["t", "o", "h", "l", "c", "v"]
columns = ["Date", "Open", "High", "Low", "Close", "Volume"]

#___________________________________________*Get Stocks Json files*________________________________________________________
class StockDataCatcher():
    def __init__(self, url, monthBefor, fromDate, toDate=time.time(), resolution="D"):
        self.url = url
        self.monthBefor = monthBefor
        self.fromDate = fromDate
        self.toDate = toDate
        self.resolution = resolution
    
    def pastDate(self, month):
        time = datetime.datetime.now()
        month_count = month
        relative_month = time - relativedelta(months=month_count)
        relative_month_timestamp = datetime.datetime.timestamp(relative_month)
        return relative_month_timestamp

    def getAllStockTickerData(self, stockTicker):
        from_date = int(self.pastDate(self.monthBefor))
        to_date = int(self.pastDate(0))
        url =  self.url+"symbol=" + str(stockTicker) + "&resolution=" + str(self.resolution) + "&from=" + str(from_date) + "&to=" + str(to_date)
        data = requests.get(url=str(url))
        json_file = data.json()
        return json_file

#_____________________________________*Get companies Json files*______________________________________________________________
class CompaniesDataCatcher():
    def __init__(self,url):
        self.url = url

    def getCompanies(self):
        data = requests.get(url=str(self.url))
        json_file = data.json()
        return json_file

#_____________________________________*companies files*______________________________________________________________
class DataWrangler():
    def __init__(self, jsonData, defualtDataFormat="csv", isStockData=True):
        self.defualtDataFormat = defualtDataFormat
        self.jsonData = jsonData
        self.isStockData = isStockData

    def saveJson(self, dirName, disFileName, disParrentDir=os.getcwd()):
        #_________________________create Stock Data csv
        if self.isStockData:
            if not os.path.exists(disParrentDir+"/"+dirName):
                os.makedirs(disParrentDir+"/"+dirName)

            path = disParrentDir + "/" + dirName
            os.chdir(path)
            df = pd.DataFrame(columns=columns)

            for i in range(0,len(self.jsonData)-1):
                if stockInfoCol[i] == "t":
                    strTimeStampList = self.jsonData[str(stockInfoCol[i])]
                    jalaliTimeStamp = []
                    for time in strTimeStampList:
                        t = JalaliDate.fromtimestamp(int(time)).strftime("%Y/%m/%d")
                        jalaliTimeStamp.append(t)
                    df[columns[i]] = jalaliTimeStamp
                else:
                    df[columns[i]] = self.jsonData[str(stockInfoCol[i])]
            file_name = "{}.csv".format(disFileName)
            df.to_csv(file_name, encoding='utf-8')
        #_________________________________create Company Data csv
        else:
            if not os.path.exists(disParrentDir+"/"+dirName):
                os.makedirs(disParrentDir+"/"+dirName)

            path = disParrentDir+"/"+dirName
            os.chdir(path)
            json = self.jsonData['items']
            df = pd.DataFrame(columns=keys)

            for index, companyInfo in enumerate(json):
                tempDic = {}
                for key in keys:
                    nestedDic = {key:companyInfo[key]}
                    tempDic.update(nestedDic)
                df = df.append(tempDic , ignore_index=True)
            
            file_name = "{}.csv".format(disFileName)
            df.to_csv(file_name, encoding='utf-8')

#_____________________________________data links__________________________________________________________
def getData(serviceProviderName, startAtMonthAgo):
    companyJsonData = []
    #_____________create companies files
    for indexI,link in enumerate(links):
        print("Creating {0} Companies Files ... ".format(linksName[indexI]) )
        companies = CompaniesDataCatcher(link)
        companyJsonData.append(companies.getCompanies())
        currentDir = os.getcwd()
        wrangler = DataWrangler(jsonData=companyJsonData[indexI], isStockData = False)
        wrangler.saveJson(disParrentDir=os.getcwd(), dirName="companies/", disFileName=str(linksName[indexI]))
        os.chdir(currentDir)

    print("Companies Files Created!\n")
    curDir = os.getcwd()

    #_____________create StockData files
    for index,link in enumerate(linksName):
        print("__________Stocks Data for {0} Companies _________".format(linksName[index]))
        i = 0
        for stockTicker in companyJsonData[index]['items']:
            i += 1
            print("{0} Stock Data [{1} / {2}] ".format(linksName[index] , i, len(companyJsonData[index]['items'])))
            stockData = StockDataCatcher(stock_data_service_providers_with_bURL[serviceProviderName], startAtMonthAgo,time.time(), time.time())
            json = stockData.getAllStockTickerData(stockTicker=stockTicker['name'])
            if json['s'] != "no_data":
                wrangler = DataWrangler(jsonData=json, isStockData=True)
                wrangler.saveJson(disParrentDir=os.getcwd(), dirName="StockData/{}/".format(str(linksName[index])),disFileName=str(stockTicker['id']))
            os.chdir(curDir)
#________________________________________________________________________________________________________________
def main():
    getData(serviceProviderName='sahamyab', startAtMonthAgo=1)#serviceProviderName:'sahamyab','orshan','tsetmc'

if __name__ == '__main__':
        main()
